# Regionalização de Bacias usando Redes Neurais

## Importing Packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r /content/drive/"My Drive"/DEHA/RegBaciasCE /content

In [3]:
import sys

sys.path.append("RegBaciasCE/src")
sys.path.append("RegBaciasCE/database")

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os

In [5]:
pd.options.display.max_columns = 999

import warnings

warnings.filterwarnings('ignore')
from funcoes_auxiliares import *



In [6]:
print("GPU: ", tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU:  True


In [7]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())
!nvidia-smi

Thu Sep 17 14:48:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    69W / 149W |    134MiB / 11441MiB |      1%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
dados = pd.read_csv("RegBaciasCE/database/DadosFFN2.csv")
#dados = dados[dados['Estações ANA'] != "36220000"]
dados = dados.loc[dados.Q_0.notnull()]
dados.head()

,Estações ANA,Área de contribuição da estação fluviométrica - A (km2),Comprimento total de drenagem - CTD (km),Curve number médio – CN (mm),Parcela da bacia no cristalino - Cr,E_0,P_2,P_1,P_0,Q_2,Q_1,Q_0
0,34730000,0.019908,0.011994,0.0,0.0,0.377420,0.333448,0.221624,0.233278,0.006548,0.005097,3.870129
1,34730000,0.019908,0.011994,0.0,0.0,0.432495,0.221624,0.233278,0.020416,0.005097,0.003214,2.136953
2,34730000,0.019908,0.011994,0.0,0.0,0.597038,0.233278,0.020416,0.004953,0.003214,0.001774,1.040503
3,34730000,0.019908,0.011994,0.0,0.0,0.767287,0.020416,0.004953,0.001582,0.001774,0.000864,0.629442
4,34730000,0.019908,0.011994,0.0,0.0,0.863514,0.004953,0.001582,0.003185,0.000864,0.000523,0.388273


In [9]:
bacias = list(set(dados['Estações ANA']))
X, Y = dados.drop(['Estações ANA','Q_0'],axis=1).values,dados['Q_0'].values

In [10]:
# Parâmetros
learning_rate = 0.001
training_epochs = 400
batch_size = 32  # Diminua o tamanho dos lotes se você não tiver memória suficiente

n_input = X.shape[1]
n_classes = 1

n_h1 = 50
n_h2 = 50
n_h3 = 10
n_h4 = 10
n_h5 = 1


In [11]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

main_input = Input(shape = (n_input,),dtype='float32', name='main_input')
x = Dense(n_h1,activation ='relu',name='hidden_layer1')(main_input)
#x = Dropout(0.1)(x)
x = Dense(n_h2,activation ='relu',name='hidden_layer2')(x)
#x = Dropout(0.1)(x)

y = Dense(1,activation ='linear',name='streamflow')(x)


np.random.seed(1500)
model = Model(inputs=[main_input], outputs=[y])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 10)]              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 50)                550       
_________________________________________________________________
hidden_layer2 (Dense)        (None, 50)                2550      
_________________________________________________________________
streamflow (Dense)           (None, 1)                 51        
Total params: 3,151
Trainable params: 3,151
Non-trainable params: 0
_________________________________________________________________


In [14]:
#!mkdir /content/RegBaciasCE/src/models/FFN2/CrossValid
model.save_weights('RegBaciasCE/src/models/FFN2/CrossValid/random.weights.FFN2.hdf5')

In [15]:
nome, n_obs,nse, cor, rmse = [], [], [], [], []

for bacia in bacias:
    idx = dados['Estações ANA'] == bacia

    X_treino, Y_treino = X[~idx], Y[~idx]
    X_teste, Y_teste = X[idx], Y[idx]





    model.load_weights('RegBaciasCE/src/models/FFN2/CrossValid/random.weights.FFN2.hdf5')


    model.compile(loss=[RMSE_Keras],optimizer ='RMSprop',
                metrics=['accuracy'])
    model.fit(X_treino, Y_treino,epochs=training_epochs,
            batch_size=batch_size,verbose=0, validation_split=0.1)


    model.save_weights('RegBaciasCE/src/models/FFN2/CrossValid/weights.Model2_' +  str(bacia) + '.hdf5')

    Y_ = model.predict(X_teste)

    Y_Prev = Y_
    for i in range(Y_Prev.shape[0]):
        if Y_Prev[i] < 0:
            Y_Prev[i] = 0
        #Y_Prev = No_Neg(Y_Prev)

    
    Y_test = np.expand_dims(Y_teste,1)

    k_ = np.mean(Y_test) / np.mean(Y_Prev)

    nome.append(bacia)
    cor.append(np.corrcoef(Y_Prev.T,Y_test.T)[1][0])
    rmse.append(RMSE(Y_test,Y_Prev))
    nse.append(NASH(Y_test,Y_Prev)[0])
    n_obs.append(len(Y_teste))


    print(pd.DataFrame({"Bacia":nome, "Observacoes":n_obs, "NSE":nse,"Cor": cor,"RMSE": rmse}))  


      Bacia  Observacoes       NSE      Cor       RMSE
0  36160000          937  0.773983  0.88356  42.930404
      Bacia  Observacoes       NSE       Cor       RMSE
0  36160000          937  0.773983  0.883560  42.930404
1  35125000          309  0.690024  0.966512  10.846004
      Bacia  Observacoes       NSE       Cor       RMSE
0  36160000          937  0.773983  0.883560  42.930404
1  35125000          309  0.690024  0.966512  10.846004
2  34730000          443 -1.930679  0.496535   8.598631
      Bacia  Observacoes       NSE       Cor       RMSE
0  36160000          937  0.773983  0.883560  42.930404
1  35125000          309  0.690024  0.966512  10.846004
2  34730000          443 -1.930679  0.496535   8.598631
3  35050000           95  0.632777  0.834377   5.336975
      Bacia  Observacoes       NSE       Cor       RMSE
0  36160000          937  0.773983  0.883560  42.930404
1  35125000          309  0.690024  0.966512  10.846004
2  34730000          443 -1.930679  0.496535   8.5

In [16]:
results = pd.DataFrame({"Bacia":nome, "Observacoes":n_obs, "NSE":nse,"Cor": cor,"RMSE": rmse})
results.to_csv("RegBaciasCE/src/models/FFN2/ResultsFFN2.csv",index=False)

In [17]:
!cp -r /content/RegBaciasCE/src/models/FFN2  /content/drive/"My Drive"/DEHA/RegBaciasCE/src/models/